# Part 4: Federated Learning with Remote Gradient Averaging

In Part 2 of this tutorial, we trained a model using a very simple version of Federated Learning. This required each data owner to trust the model owner to be able to see their gradients. In this next tutorial, we'll show how to use the advanced aggregation tootls from Part 3 to allow the gradients to be aggregated by a trusted "secure worker" before the final result is sent back to the model owner (us). In this way, only the secure worker can see whose gradient came from whom. We might be able to tell which parts of the model changed, but we do NOT know which worker (bob or alice) made which change, which creates a layer of privacy

In [1]:
import syft as sy
import copy
hook = sy.TorchHook()
from torch import nn, optim

# Step 1: Create Data Owners

First, we're going to create two data owners (Bob and Alice) each with a small amount of data.

In [2]:
# create a couple workers

bob = sy.VirtualWorker(id="bob")
alice = sy.VirtualWorker(id="alice")
secure_worker = sy.VirtualWorker(id="secure_worker")

bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

# A Toy Dataset
data = sy.Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = sy.Var(sy.FloatTensor([[0],[0],[1],[1]]))

# get pointers to training data on each worker by
# sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

# Create Our Model

For this example, we're going to train with a simple Linear model

In [3]:
# Iniitalize A Toy Model
model = nn.Linear(2,1)


# Send a Copy of the Model to Alice and Bob

Next, we need to send a copy of the current model to Alice and Bob so that they can perform steps of learning on their own datasets.

In [4]:
bobs_model = model.copy().send(bob)
alices_model = model.copy().send(alice)

bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

# Train Bob's and Alice's Models (in parallel) for a While

In [5]:
# Train Bob's Model
bobs_opt.zero_grad()
bobs_pred = bobs_model(bobs_data)
bobs_loss = ((bobs_pred - bobs_target)**2).sum()
bobs_loss.backward()

bobs_opt.step()
bobs_loss = bobs_loss.get().data[0]

# Train Alice's Model
alices_opt.zero_grad()
alices_pred = alices_model(alices_data)
alices_loss = ((alices_pred - alices_target)**2).sum()
alices_loss.backward()

alices_opt.step()
alices_loss = alices_loss.get().data[0]
alices_loss

1.0242788791656494

# Send Both Updated Models to a Secure Worker (directly)

Note that this use of our API means that each model is sent DIRECTLY to the secure_worker. We never see it.

In [6]:
alices_model.move(secure_worker)
bobs_model.move(secure_worker)

Linear(in_features=2, out_features=1, bias=True)

# Average the Models

In [7]:
model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
""

''

# Rinse and Repeat

And now we just need to iterate this multiple times!

In [8]:
iterations = 10
worker_iters = 5

for a_iter in range(iterations):
    
    bobs_model = model.copy().send(bob)
    alices_model = model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(),lr=0.1)

    for wi in range(worker_iters):

        # Train Bob's Model
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()

        bobs_opt.step()
        bobs_loss = bobs_loss.get().data[0]

        # Train Alice's Model
        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred - alices_target)**2).sum()
        alices_loss.backward()

        alices_opt.step()
        alices_loss = alices_loss.get().data[0]
    
    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)
    
    model.weight.data.set_(((alices_model.weight.data + bobs_model.weight.data) / 2).get())
    model.bias.data.set_(((alices_model.bias.data + bobs_model.bias.data) / 2).get())
    
    print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:0.007271846290677786 Alice:0.02405245415866375
Bob:0.008776296861469746 Alice:0.010423511266708374
Bob:0.009437767788767815 Alice:0.005108115263283253
Bob:0.009096330963075161 Alice:0.002636871999129653
Bob:0.00817464292049408 Alice:0.0014180572470650077
Bob:0.007018323987722397 Alice:0.0007943799719214439
Bob:0.005842213053256273 Alice:0.00046386124449782073
Bob:0.004759777337312698 Alice:0.000282174936728552
Bob:0.0038191911298781633 Alice:0.00017843986279331148
Bob:0.0030310030560940504 Alice:0.00011690950486809015
